# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [13]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
time_steps_per_day = 6
batch_size = 2

sim_dirs = [
    ('NYC_Heat_Test', [
        'NYC_summer_2000_01p',
        'NYC_summer_2010_25p',
        'NYC_summer_2015_50p',
        'NYC_summer_2017_75p',
        'NYC_summer_2018_99p'
    ]),
    ('PHX_Heat_Test', [
        'PHX_summer_2008_25p',
        'PHX_summer_2009_50p',
        'PHX_summer_2011_99p',
        'PHX_summer_2015_75p',
        'PHX_summer_2020_01p'
    ])
]

sim_names = []
for sim_dir, subdirs in sim_dirs:
    for subdir in subdirs:
        sim_names.append(sim_dir + '/' + subdir)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Create the training dataset using create_atmo_dataset with sim_names
client = storage.Client(project="climateiq")
ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    shuffle=None,
    timesteps_per_day=time_steps_per_day,
    max_blobs=None
).batch(batch_size=batch_size)
train_dataset, val_dataset, test_dataset = dataset.split_dataset(
    ds, train_frac=0.7, val_frac=0.15, test_frac=0.15
)

2024-12-02 20:58:12.893772: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 1 of 1000
2024-12-02 20:58:14.439274: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2024-12-02 20:58:14.440822: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/josiahkp/climateiq-cnn/usl_models/usl_mode

In [8]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)
# model._model.summary()

In [9]:
# Train the model
model.fit(train_dataset, epochs=1)

Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)
Split shape: (None, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (None, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (None, 50, 50, 8, 256)


2024-12-02 20:46:21.683588: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm/conv_lstm/conv_lstm2d/while/body/_1/atmo_conv_lstm/conv_lstm/conv_lstm2d/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Output shape: TensorShape([None, 8, 50, 50, 256])


2024-12-02 20:46:22.551002: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900


      1/Unknown - 11s 11s/step - loss: 15631.8096 - mean_absolute_error: 63.2611 - root_mean_squared_error: 125.0272Output shape: TensorShape([None, 8, 50, 50, 256])
      2/Unknown - 11s 120ms/step - loss: 12389.4902 - mean_absolute_error: 50.5707 - root_mean_squared_error: 111.3081Output shape: TensorShape([None, 8, 50, 50, 256])
2/2 [==============================] - 11s 227ms/step - loss: 12389.4902 - mean_absolute_error: 50.5707 - root_mean_squared_error: 111.3081


In [10]:
inputs, labels = next(iter(train_dataset))
{key: tensor.shape for key, tensor in inputs.items()}

{'spatiotemporal': TensorShape([2, 6, 200, 200, 12]),
 'spatial': TensorShape([2, 200, 200, 22]),
 'lu_index': TensorShape([2, 200, 200])}

In [11]:
model._model.summary()

Model: "atmo_conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  488       
                                                                 
 spatial_cnn (Sequential)    (None, 50, 50, 128)       252992    
                                                                 
 spatiotemporal_cnn (Sequen  (None, None, 50, 50, 64   30480     
 tial)                       )                                   
                                                                 
 conv_lstm (Sequential)      (None, None, 50, 50, 51   45877248  
                             2)                                  
                                                                 
 t2_output_cnn (Sequential)  (None, None, 200, 200,    69729     
                             1)                                  
                                                    

In [12]:
# Test calling the model on some training data
inputs, labels = next(iter(train_dataset))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)

Split shape: (2, 50, 50, 5, 2, 256)
Separated shape (pre-slice): (2, 50, 50, 10, 256)
Reduced shape after removing outermost time steps: (2, 50, 50, 8, 256)
Output shape: TensorShape([2, 8, 50, 50, 256])
Prediction shape: (2, 8, 200, 200, 5)
